# Max Bin Names per namespace
Use this code to test. Current server 7.x
Maximum bin names per namespace:
- Server 7: unlimited.
- Server 5 and 6: 65,535.
- Server 4 and earlier: 32,767.

### Housekeeping - Setup & wipe out any prior records on the Aerospike Server

We have a namespace **_test_** pre-defined on the server. Lets truncate it using _asadm_.

This is needed while doing code development. If you want to clear the iJava Kernel of all Java objects and run all cells from scratch, Kernel->Restart & Run All, this will ensure any records written on the underlying Aerospike cluster are purged.

First, we need required imports for using %sh in interactive Java Kernel. (This is specific to the iJava Kernel implementation by **Spencer Park** that we are using.)

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

### Running _asadm_ in iJava
We can run _asadm_ commands inline. Below, we will use the truncation command, which normally requires an interactive confirmation, which we will skip by using the _--no-warn_ flag. No output will be displayed.

In [2]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

#### Add Java Client POM Dependency  
Jupyter Notebook way!

In [3]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

####  Java Client Imports and Test code

In [4]:
//Required Imports
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;

//import com.aerospike.client.cdt.ListPolicy;
//import com.aerospike.client.cdt.ListOperation;
//import com.aerospike.client.cdt.ListOrder;
//import com.aerospike.client.cdt.ListWriteFlags;
//import com.aerospike.client.cdt.ListReturnType;

//import com.aerospike.client.cdt.MapPolicy;
//import com.aerospike.client.cdt.MapOrder;
//import com.aerospike.client.cdt.MapWriteFlags;
//import com.aerospike.client.cdt.MapReturnType;

//import com.aerospike.client.exp.MapExp;
//import com.aerospike.client.cdt.CTX;
//import java.util.HashMap;
//import java.util.Map;

import com.aerospike.client.Operation;
import com.aerospike.client.query.Statement;
import com.aerospike.client.exp.Exp;
//import com.aerospike.client.cdt.MapOperation;

//import com.aerospike.client.policy.BatchPolicy;

//import java.util.Random;



In [5]:
AerospikeClient client = new AerospikeClient("127.0.0.1", 3000);


In [6]:
//Insert a record using a class based construct
class userProfile {
  public void createProfile (Key user, String binName, String name, boolean bCheck) {    
    Bin bName = new Bin(binName, name);
    WritePolicy wPolicy1 = new WritePolicy();
    client.put(wPolicy1, user, bName);
    if(bCheck){
      System.out.println(client.get(null, user));
    }   
  }
}

In [7]:
userProfile pf = new userProfile();
int keyId = 0;
Key user = new Key("test", "testset", keyId); 
pf.createProfile(user, "bin0", "bin0", true);

(gen:1),(exp:450900550),(bins:(bin0:bin0))


In [10]:
//int keyId = 0;
//for(int i=0; i<5; i++){
for(int i=0; i<10000; i++){  
    client.delete(null, user);
    String var = "bin"+i;
    //Key user = new Key(ns, setName, keyId); 
    pf.createProfile(user, var, var, false);
}
System.out.println(client.get(null, user));

(gen:1),(exp:450900607),(bins:(bin100404:bin100404))


# Cleanup

In [11]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"